In [8]:
# --- Imports ---
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import make_scorer, roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# --- Custom Pearson scorer ---
def pearson_corr_func(estimator, X, y):
    y_pred = estimator.predict_proba(X)[:, 1]
    print(estimator.predict_proba(X)[:,1])
    corr, _ = pearsonr(y_pred, y)
    return corr

pearson_scorer = make_scorer(pearson_corr_func, response_method="predict_proba")

# --- Generate toy dataset ---
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=5,
    n_redundant=2, n_classes=2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Define model and search space ---
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, solver="saga"))
])

search_space = {
    "clf__C": Real(1e-5, 10, prior="log-uniform"),
    "clf__penalty": Categorical(["l1", "l2"]),
}

# --- Run BayesSearchCV using Pearson R ---
opt = BayesSearchCV(
    estimator=pipe,
    search_spaces=search_space,
    n_iter=10,
    cv=3,
    scoring=pearson_corr_func,  # <-- Pearson correlation
    n_jobs=-1,
    verbose=1,
    random_state=42,
)

opt.fit(X_train, y_train)

print("Best Params:", opt.best_params_)
print("Best CV Pearson R:", opt.best_score_)

# --- Evaluate on hold-out test set ---
y_pred_test = opt.best_estimator_.predict_proba(X_test)[:, 1]
pearson_r, _ = pearsonr(y_pred_test, y_test)
auc = roc_auc_score(y_test, y_pred_test)

print(f"Test Pearson R: {pearson_r:.3f}")
print(f"Test AUC: {auc:.3f}")


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[0.37935376 0.45755349 0.45129893 0.44722303 0.42459831 0.33944972
 0.50131138 0.39545445 0.5436041  0.53161974 0.5748348  0.55404437
 0.56482354 0.28498369 0.56097614 0.34175272 0.54558067 0.52209204
 0.41336581 0.31435656 0.5444149  0.52325201 0.56807964 0.48788225
 0.5832425  0.38294201 0.44054348 0.42474828 0.58115358 0.35130533
 0.56905733 0.53769818 0.42745529 0.5630515  0.6526149  0.4757363
 0.65698678 0.62277652 0.64831695 0.53884039 0.5079729  0.52303663
 0.54560412 0.51944002 0.40372708 0.44532611 0.51661255 0.47914239
 0.61817654 0.51006676 0.52384204 0.47509624 0.45942701 0.5747653
 0.47738167 0.48685162 0.57648824 0.58802097 0.41766586 0.41890127
 0.54278502 0.57515357 0.6141657  0.43924082 0.44680751 0.54064133
 0.53139257 0.42067306 0.63168188 0.48818007 0.5896203  0.52192399
 0.58563241 0.48142664 0.55147731 0.57091709 0.563093   0.49247471
 0.54483197 0.49573637 0.54223231 0.56826447 0.37967018 0.41715736
 0.56

In [14]:
# ---------------------------
# 1) Load dependencies and data
# ---------------------------
import logging
import numpy as np
import pandas as pd
import joblib, os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.frozen import FrozenEstimator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from joblib import Parallel, delayed


# Logging
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", level=logging.INFO)
logger = logging.getLogger(__name__)

# Load sample dataset (smaller or subset for testing)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("PyTorch path:", torch.__path__ )

logger.info("Loading dataset...")
chunksize = 100
list_of_dataframes = []
for df in pd.read_csv('DarpaQCGenoPheno.csv', chunksize=chunksize, index_col=0):
    list_of_dataframes.append(df)
df = pd.concat(list_of_dataframes)

ids = df["ID"].values
ax_columns = [col for col in df.columns if col.startswith('AX')]
X = df[ax_columns]
y = df["Status"]
X = X.to_numpy()
y = y.to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X)

models = ["LR", "RF", "GB"]

2025-09-03 15:33:26,110 - INFO - Loading dataset...


Using device: cuda
PyTorch path: ['/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/torch']


In [15]:
# ---------------------------
# 2) Define search spaces and run Bayesian optimization (smaller/fewer iterations)
# ---------------------------

def get_small_search_spaces():
    return {
        "LR": (
            LogisticRegression(max_iter=200, solver="saga"),
            {
                "C": Real(1e-2, 1.0, prior="log-uniform"),
                "penalty": Categorical(["l1", "l2"]),
            },
        ),
        "RF": (
            RandomForestClassifier(n_jobs=-1),
            {
                "n_estimators": Integer(50, 200),
                "max_depth": Integer(2, 10),
            },
        ),
        "GB": (
            XGBClassifier(
                tree_method="hist",  
                device="cuda",
                eval_metric="logloss",
            ),
            {
                "n_estimators": Integer(50, 200),
                "max_depth": Integer(2, 6),
                "learning_rate": Real(0.05, 0.3, prior="log-uniform"),
            },
        ),
        # "MLP": (
        #     MLPClassifier(max_iter=200),
        #     {
        #         "hidden_layer_sizes": Categorical([32, 64, (32, 16), (64, 32), (128,)]),
        #         "alpha": Real(1e-5, 1e-2, prior="log-uniform"),
        #         "learning_rate_init": Real(1e-3, 1e-2, prior="log-uniform"),
        #     },
        # ),
    }

def tune_model(X, y, model_name, n_iter=5):
    base_model, search_space = get_small_search_spaces()[model_name]
    logger.info(f"Bayesian optimization for {model_name}")
    opt = BayesSearchCV(
        estimator=base_model,
        search_spaces=search_space,
        n_iter=n_iter,
        cv=3,
        scoring="roc_auc",
        n_jobs=-1,
        verbose=0,
    )
    opt.fit(X, y)
    logger.info(f"Best {model_name} params: {opt.best_params_}")
    return opt.best_estimator_

tuned_models = {name: tune_model(X, y, name) for name in models}


2025-09-03 15:34:06,832 - INFO - Bayesian optimization for LR
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/

In [16]:
# ---------------------------
# 3) Run simplified cross-validation (3 folds instead of 10)
# ---------------------------
skf_outer = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

results = []
all_preds = []

for fold, (train_val_idx, test_idx) in enumerate(skf_outer.split(X, y)):
    logger.info(f"Outer Fold {fold+1}/3")

    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y[train_val_idx], y[test_idx]
    ids_test = ids[test_idx]

    X_train, X_cal, y_train, y_cal = train_test_split(
        X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
    )

    fold_df = pd.DataFrame({"ID": ids_test, "true_label": y_test, "fold": fold+1})

    for name, tuned_model in tuned_models.items():
        tuned_model.fit(X_train, y_train)
        frozen = FrozenEstimator(tuned_model)
        calibrated = CalibratedClassifierCV(frozen, method="isotonic", cv="prefit")
        calibrated.fit(X_cal, y_cal)

        probs = calibrated.predict_proba(X_test)[:, 1]
        fold_df[name] = probs

        auc = roc_auc_score(y_test, probs)
        logger.info(f"{name} Fold {fold+1} AUC={auc:.3f}")

    all_preds.append(fold_df)

prob_df = pd.concat(all_preds, axis=0).sort_values("ID")
prob_df.head()


2025-09-03 15:50:01,640 - INFO - Outer Fold 1/3
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
2025-09-03 15:52:21,120 - INFO - LR Fold 1 AUC=0.619
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
2025-09-03 15:52:22,055 - INFO - RF Fold 1 AUC=0.583
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-

ID  true_label  fold        LR        RF       GB
29     B-1           1     2  0.495726  0.535270  0.48538
0   B-1000           0     1  0.208333  0.542857  0.40678
0   B-1002           0     2  0.495726  0.535270  0.48538
1   B-1003           0     2  0.495726  0.272727  0.48538
2   B-1005           0     2  0.495726  0.535270  0.48538

In [20]:
# ---------------------------
# 4) Save model output
# ---------------------------
# Save predictions
# prob_df.to_csv("debug_predicted_probabilities.csv", index=False)
logger.info("Saved debug predictions to debug_predicted_probabilities.csv")

# Save models
os.makedirs("models/debug", exist_ok=True)
for name, model in tuned_models.items():
    path = os.path.join("models/debug/", f"{name}_debug_model.joblib")
    joblib.dump(model, path)
    logger.info(f"Saved {name} model to {path}")

prob_df

2025-09-03 15:58:12,351 - INFO - Saved debug predictions to debug_predicted_probabilities.csv
2025-09-03 15:58:12,367 - INFO - Saved LR model to models/debug/LR_debug_model.joblib
2025-09-03 15:58:12,422 - INFO - Saved RF model to models/debug/RF_debug_model.joblib
2025-09-03 15:58:12,433 - INFO - Saved GB model to models/debug/GB_debug_model.joblib


ID  true_label  fold        LR        RF        GB
29      B-1           1     2  0.495726  0.535270  0.485380
0    B-1000           0     1  0.208333  0.542857  0.406780
0    B-1002           0     2  0.495726  0.535270  0.485380
1    B-1003           0     2  0.495726  0.272727  0.485380
2    B-1005           0     2  0.495726  0.535270  0.485380
..      ...         ...   ...       ...       ...       ...
782   Y_988           1     3  0.396552  0.432836  0.491803
785   Y_991           1     2  0.571429  0.535270  0.538462
783   Y_992           0     3  0.576642  0.531746  0.602941
784   Y_995           1     3  0.402923  0.700000  0.727273
785   Y_998           1     1  0.684211  0.585366  0.709091

[2357 rows x 6 columns]

In [1]:
## testing 9/5

import os
from datetime import datetime
import json

# Simulate tuned parameters for testing
tuned_params = {
    "LR": {"C": 0.1, "penalty": "l2"},
    "RF": {"n_estimators": 500, "max_depth": 10},
    "GB": {"n_estimators": 300, "max_depth": 5, "learning_rate": 0.1},
}

# Create unique run folder based on today's date
today_str = datetime.now().strftime("%b%d").lower()  # e.g., "sep05"
run_dir = os.path.join("models", today_str)
os.makedirs(run_dir, exist_ok=True)

# Save dummy hyperparameters to JSON file in that folder
json_path = os.path.join(run_dir, "best_hyperparams.json")
with open(json_path, "w") as f:
    json.dump(tuned_params, f, indent=4)

# Verify the file path and show contents
(json_path, os.path.exists(json_path))



('models/sep05/best_hyperparams.json', True)